# Shapley-Value

In [1]:
import IPythonImporter
import GameFramework as gf

import numpy as np
import scipy.stats as stats

importing Jupyter notebook from GameFramework.ipynb


In [2]:
def shapley(scenario):
    
    # data structur: | Skill | CostFunction | Location | Agent |
    struc = np.empty((0,2));
    
    # Alle SkillCapacities aufsplitten
    for agent in scenario.agents:
        for skill in agent.skillCaps:
            struc = np.concatenate((struc, [[str(skill.skill), str(agent._id)] \
                                     for i in range(skill.capacity)]), axis=0)
            
    # Alle möglichen Koalitionen durchiterieren, codiert als binäre Zahl
    bit_digits = struc.shape[0]
    int_number = int(pow(2, bit_digits-1) + pow(2, bit_digits-1)-1)

    for i in xrange(int_number+1):
        bit_string = list(str(bin(i)[2:].zfill(bit_digits)))
        bit_string = np.reshape(np.repeat(bit_string, 2), (bit_digits, 2))
        
        coalition = struc[bit_string=='1']
        coalition = np.reshape(coalition, ((coalition.shape[0]/2, 2)))
        
        # Check: Kann die Koalition eine oder mehrere Baustellen bedienen?
        if len(coalition) > 0:
            coalitionAggr = stats.itemfreq(coalition[:,0])
            possibleSites = list()

            for site in scenario.sites:  # für jede Baustelle
                valid = True
                
                for req in site.requests:  # Genug Skills in Coalition für Baustelle?
                    skillIdx = np.where(coalitionAggr[:,0] == str(req.skill))

                    if  coalitionAggr[:,1][skillIdx].shape[0] == 0 or \
                       (coalitionAggr[:,1][skillIdx]).astype(int) < req.amount:
                        valid = False
                        break
                        
                if valid == True:
                    possibleSites.append(site._id)

            
scenario = gf.Scenario( agentCount=2, siteCount=2, skills=[gf.Skill("mauern"), gf.Skill("baggern")], \
                        skillCap=[[5,3],[3,5]], skillReq=[[7,15],[8,8]], budget=[9000000], \
                        costFunction=[['lambda k,d:(k/5+1)*d+200*k', 'lambda k,d:(k/3+1)*d+250*k']] )
    
if __name__=='__main__':
    from timeit import Timer
    t = Timer(lambda: shapley(scenario))
    print t.timeit(number=1)

3.89228606224
